In [45]:
# Intialization
import os
import sys

os.environ["SPARK_HOME"] = "/home/talentum/spark"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
# In below two lines, use /usr/bin/python2.7 if you want to use Python 2
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3.6" 
os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/bin/python3"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

# NOTE: Whichever package you want mention here.
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0 pyspark-shell' 
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.3 pyspark-shell'
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'

In [46]:
#Entrypoint 2.x
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().getOrCreate()

# On yarn:
# spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().master("yarn").getOrCreate()
# specify .master("yarn")

sc = spark.sparkContext

In [47]:
df = spark.read.csv('realtor-data.zip.csv', header=True, inferSchema=True)

In [48]:
df.show(5)

+-----------+--------+--------+---+----+--------+---------+----------+-----------+--------+----------+--------------+
|brokered_by|  status|   price|bed|bath|acre_lot|   street|      city|      state|zip_code|house_size|prev_sold_date|
+-----------+--------+--------+---+----+--------+---------+----------+-----------+--------+----------+--------------+
|   103378.0|for_sale|105000.0|  3|   2|    0.12|1962661.0|  Adjuntas|Puerto Rico|     601|     920.0|          null|
|    52707.0|for_sale| 80000.0|  4|   2|    0.08|1902874.0|  Adjuntas|Puerto Rico|     601|    1527.0|          null|
|   103379.0|for_sale| 67000.0|  2|   1|    0.15|1404990.0|Juana Diaz|Puerto Rico|     795|     748.0|          null|
|    31239.0|for_sale|145000.0|  4|   2|     0.1|1947675.0|     Ponce|Puerto Rico|     731|    1800.0|          null|
|    34632.0|for_sale| 65000.0|  6|   2|    0.05| 331151.0|  Mayaguez|Puerto Rico|     680|      null|          null|
+-----------+--------+--------+---+----+--------+-------

In [49]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Realtor Data").getOrCreate()

housing = spark.read.csv("realtor-data.zip.csv", header=True, inferSchema=True)

In [50]:
housing.show(5)

+-----------+--------+--------+---+----+--------+---------+----------+-----------+--------+----------+--------------+
|brokered_by|  status|   price|bed|bath|acre_lot|   street|      city|      state|zip_code|house_size|prev_sold_date|
+-----------+--------+--------+---+----+--------+---------+----------+-----------+--------+----------+--------------+
|   103378.0|for_sale|105000.0|  3|   2|    0.12|1962661.0|  Adjuntas|Puerto Rico|     601|     920.0|          null|
|    52707.0|for_sale| 80000.0|  4|   2|    0.08|1902874.0|  Adjuntas|Puerto Rico|     601|    1527.0|          null|
|   103379.0|for_sale| 67000.0|  2|   1|    0.15|1404990.0|Juana Diaz|Puerto Rico|     795|     748.0|          null|
|    31239.0|for_sale|145000.0|  4|   2|     0.1|1947675.0|     Ponce|Puerto Rico|     731|    1800.0|          null|
|    34632.0|for_sale| 65000.0|  6|   2|    0.05| 331151.0|  Mayaguez|Puerto Rico|     680|      null|          null|
+-----------+--------+--------+---+----+--------+-------

In [51]:
#Get information about the data
housing.printSchema()

root
 |-- brokered_by: double (nullable = true)
 |-- status: string (nullable = true)
 |-- price: double (nullable = true)
 |-- bed: integer (nullable = true)
 |-- bath: integer (nullable = true)
 |-- acre_lot: double (nullable = true)
 |-- street: double (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- zip_code: integer (nullable = true)
 |-- house_size: double (nullable = true)
 |-- prev_sold_date: timestamp (nullable = true)



In [52]:
housing.describe().show()

+-------+------------------+--------+-----------------+------------------+------------------+-----------------+------------------+--------------------+-------+-----------------+-----------------+
|summary|       brokered_by|  status|            price|               bed|              bath|         acre_lot|            street|                city|  state|         zip_code|       house_size|
+-------+------------------+--------+-----------------+------------------+------------------+-----------------+------------------+--------------------+-------+-----------------+-----------------+
|  count|           2221849| 2226382|          2224841|           1745065|           1714611|          1900793|           2215516|             2224975|2226374|          2226083|          1657898|
|   mean| 52939.89328662749|    null|524195.5192906774|3.2758407279958055| 2.496440300452989|15.22302744697555|1012324.9206654341|                null|   null|52186.67634135834|2714.471334786579|
| stddev|30642.75288

In [53]:
#Count missing values

#from pyspark.sql.functions import col, isnan, when, count

#housing.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in housing.columns]).show()

from pyspark.sql.functions import col, isnan, when, count
from pyspark.sql.types import DoubleType, FloatType

#function to check for nan and null values based on data type
def count_nulls(c):
    if housing.schema[c].dataType in [DoubleType(), FloatType()]:
        return count(when(isnan(col(c)) | col(c).isNull(), c)).alias(c)
    else:
        return count(when(col(c).isNull(), c)).alias(c)

#applied function to each column
null_counts = housing.select([count_nulls(c) for c in housing.columns])
null_counts.show()



+-----------+------+-----+------+------+--------+------+----+-----+--------+----------+--------------+
|brokered_by|status|price|   bed|  bath|acre_lot|street|city|state|zip_code|house_size|prev_sold_date|
+-----------+------+-----+------+------+--------+------+----+-----+--------+----------+--------------+
|       4533|     0| 1541|481317|511771|  325589| 10866|1407|    8|     299|    568484|        734297|
+-----------+------+-----+------+------+--------+------+----+-----+--------+----------+--------------+



In [55]:
#Count duplicate rows
housing.groupBy(housing.columns).count().filter(col("count") > 1).count()

0

In [56]:
#Count values in the status column
housing.groupBy("status").count().orderBy("count", ascending=False).show()

+--------------+-------+
|        status|  count|
+--------------+-------+
|      for_sale|1389306|
|          sold| 812009|
|ready_to_build|  25067|
+--------------+-------+



In [57]:
#Count non-null values in the prev_sold_date column
housing.filter(col("prev_sold_date").isNotNull()).count()

1492085

In [58]:
#Count values in the state column
housing.groupBy("state").count().orderBy("count", ascending=False).show()

+--------------+------+
|         state| count|
+--------------+------+
|       Florida|249432|
|    California|227215|
|         Texas|208335|
|      New York|103159|
|North Carolina| 85745|
|      Illinois| 85280|
|       Georgia| 80977|
|  Pennsylvania| 78373|
|       Arizona| 72825|
|      Virginia| 68763|
|    Washington| 62461|
|          Ohio| 59207|
|    New Jersey| 48199|
|      Maryland| 46052|
|      Missouri| 45145|
|     Minnesota| 43412|
|      Michigan| 42429|
|     Wisconsin| 42390|
|South Carolina| 42367|
|     Tennessee| 40964|
+--------------+------+
only showing top 20 rows



In [59]:
#Drop columns

housing = housing.drop("zip_code", "city","street")

In [60]:
#print schema
housing.printSchema()

root
 |-- brokered_by: double (nullable = true)
 |-- status: string (nullable = true)
 |-- price: double (nullable = true)
 |-- bed: integer (nullable = true)
 |-- bath: integer (nullable = true)
 |-- acre_lot: double (nullable = true)
 |-- state: string (nullable = true)
 |-- house_size: double (nullable = true)
 |-- prev_sold_date: timestamp (nullable = true)



In [62]:
#Dropping NA values from column "bed"
housing = housing.dropna(subset=["bed"])

In [65]:
#Dropping NA values from column "bath"
housing = housing.dropna(subset=["bath"])

In [68]:
#Dropping NA values from column "house_size"
housing = housing.dropna(subset=["house_size"])

In [69]:
#Dropping NA values from column "price"
housing = housing.dropna(subset=["price"])

In [72]:
#Dropping NA values from column "acre_lot"
housing = housing.dropna(subset=["acre_lot"])

In [73]:
#function to check for nan and null values based on data type
def count_nulls(c):
    if housing.schema[c].dataType in [DoubleType(), FloatType()]:
        return count(when(isnan(col(c)) | col(c).isNull(), c)).alias(c)
    else:
        return count(when(col(c).isNull(), c)).alias(c)

# applied function to each column
null_counts = housing.select([count_nulls(c) for c in housing.columns])
null_counts.show()

+-----------+------+-----+---+----+--------+-----+----------+--------------+
|brokered_by|status|price|bed|bath|acre_lot|state|house_size|prev_sold_date|
+-----------+------+-----+---+----+--------+-----+----------+--------------+
|       2260|     0|    0|  0|   0|       0|    0|         0|        270807|
+-----------+------+-----+---+----+--------+-----+----------+--------------+



In [74]:
#show count
housing.count()

1360716

In [78]:
housing.show(2000)

+-----------+--------+---------+---+----+--------+--------------+----------+-------------------+
|brokered_by|  status|    price|bed|bath|acre_lot|         state|house_size|     prev_sold_date|
+-----------+--------+---------+---+----+--------+--------------+----------+-------------------+
|   103378.0|for_sale| 105000.0|  3|   2|    0.12|   Puerto Rico|     920.0|               null|
|    52707.0|for_sale|  80000.0|  4|   2|    0.08|   Puerto Rico|    1527.0|               null|
|   103379.0|for_sale|  67000.0|  2|   1|    0.15|   Puerto Rico|     748.0|               null|
|    31239.0|for_sale| 145000.0|  4|   2|     0.1|   Puerto Rico|    1800.0|               null|
|   103378.0|for_sale| 179000.0|  4|   3|    0.46|   Puerto Rico|    2520.0|               null|
|     1205.0|for_sale|  50000.0|  3|   1|     0.2|   Puerto Rico|    2040.0|               null|
|    50739.0|for_sale|  71600.0|  3|   2|    0.08|   Puerto Rico|    1050.0|               null|
|    81909.0|for_sale| 100000.